Quantitative Value Strategy
We will try to buy stock that are cheap relative to their intrinsic value.
We will build a simple value strategy that relies on one metric (price to value ratio - stock price/ avr yearly earning). then we are going to improve that value strategy by 
creating a composiute value strategy that takes into account may different metrics wh trying to measure intrinsic value

We're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

IEX provides price to value ratio in documentation. Under "Quote" endpoint is "peRatio..." which is the price to value ratio. Under "Quote" is "latest price" as well.

In [11]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#test if API was called correctly. want to see 200
#data = requests.get(api_url)
#print(data.status_code)
data

In [12]:
pe_ratio = data['peRatio']
pe_ratio

35.02

Re-using chunck function from Momentum Strategy

In [13]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
#We take every group of stocks that's in the symbols group variable and create a comma seperated string.
#  that has all of the tickers from each list
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [22]:
#Creating a blank dataframe
#final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe

In [15]:
#testing code. should be A-Z
#for symbol_string in symbol_strings:
#    for symbol in symbol_string.split(','):
#        print(symbol)

In [17]:
#testing. shopuld print six 200's. search error if any.
#for symbol_string in symbol_strings:
#    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#    data = requests.get(batch_api_call_url) 
#    print(data.status_code)
#    for symbol in symbol_string.split(','):
#         pass



In [27]:
# looping over every symbol_string in our symbol_strings object and another loop that loops over every ticker
# in that symbol_string
for symbol_string in symbol_strings:
#     print(symbol_strings)
# in IEX under batch requests copy second link ""...?symbols=aapl,fb,tsla&types=quote,...". Changed "..symbols=aapl,fb.." to "{symbol_string}" 
# Deleted "news,chart" endpoints and "range..last..". replaced by our token. Also "cloud" changed to sandbox.          
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
# We created our batch API call. Now we will execute our HTTP request
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.52,2.39,N/A
1,NRG,39.68,2.39,N/A
2,NRG,39.55,2.46,N/A
3,BIO,617.23,4.75,N/A
4,BIO,607.00,4.81,N/A
...,...,...,...,...
550,YUM,107.28,31.68,N/A
551,ZBH,169.74,-229.2,N/A
552,ZBRA,513.58,57.28,N/A
553,ZION,57.72,24.81,N/A


Removing Glamour Stocks
A Glamour stock is the opposite of a value stock.
We will sort the dataframe such the the stocks witht he lowest "price earnings ratio are at the top and we'll delete anything that isn't the top 50.  

In [ ]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe 

the stocks with negative price to earnings ratio (price/earnings) price has to be negative
or earnings has to be negative. Stocks can't be negative therefore their earnings are negative. 
We can remove the negative ratios by: adding "...final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]" 

In [29]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
#modifying original dataframe
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [ ]:
Calculating the Number of Shares to Buy

In [32]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
portfolio_input()        

Enter the value of your portfolio:1000000


Position size can be calculated by dividing the portfolio size by the number of shares in the portfoliio. 

In [36]:
#you can take any column to divide. Not obligatory to use index.
position_size = float(portfolio_size) / len(final_dataframe.index)
#looping through every row in the dataframe and modify the value that is contained
# the 'Number of Shares to Buy' column. 
for i in range(0, len(final_dataframe['Ticker'])):
    #divide our position size by that stock's stock price. 
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.52,2.39,519
1,NRG,39.68,2.39,504
2,NRG,38.70,2.45,516
3,NRG,39.55,2.46,505
4,BIO,617.23,4.75,32
5,BIO,599.36,4.78,33
6,BIO,607.00,4.81,32
7,BIO,611.38,4.81,32
8,AIV,5.17,4.89,3868
9,AIV,5.16,4.89,3875


We will now build a better Value Stategy. We will build a strategy that takes into account many different metrics. It's makes a better because some value metrics tend to have certain flaws. ie the price to earnings ratio doesn't work tha t well with stocks that have negative earnings. Another popular evaluation metric is the "price to book ratio" and that metric doesn't perform well stocks that perform a bunch of share buy backs. 

One way to reduce flaws of any specific metric is by a creatiung value strategies that take into account a composite of values. So instead of ranking stocks on "price to earnings ratio" they rank percentiles for a bunch of different evaluation metrics and rank stocks in the average of those percentile scores.

We'll filter for stocks with the lowest percentiles on the following metrics:
   1)Price-to-earnings ratio
   2)Price-to-book ratio
   3)Price-to-sales ratio
   4)Enterprise Value divided by Earnings Before  
      Interest, Taxes, Depreciation, and Amortization    
      (EV/EBITDA) -  the proce you would have to pay to buy  
      out the company in full. Including debt and any minority 
      share holdings that exist. 
   5)Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

We are going to do a single API call and then lood through that same API call for different stocks that are listed in our stocks datastructure to build a pandas dataframe that contsins all of the data.  

To start. we're going to use one symbol, AAPL. We're also going to do a batch API call but aunlike before, we are not doing a batch API call because we want to query the IEX cloud API for multiple symbols but querying for multiple endpoints for our metrics.   

In [37]:
# testing. should return 200
# Changes to link: We replacedsymbol_string with symbol 
#symbol = 'AAPL'
#batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(batch_api_call_url)
#data.status_code

200

In [ ]:
#comparing copied API from above before and after
#batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
#batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'

In [40]:
#test
#symbol = 'AAPL'
#batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(batch_api_call_url).json()
#print(data[symbol]['quote']['peRatio'])

33.56


In [51]:
#symbol = 'AAPL'
#batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats'])
#we will use the below enpoints
#print(data[symbol]['advanced-stats']['priceToBook'])
#print(data[symbol]['advanced-stats']['peRatio'])
#print(data[symbol]['advanced-stats']['priceToSales'])

In [52]:
# Changes to link: We replaced symbol_string with symbol 
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
# in IEX search "price-to-book". copy endpoint "advanced-stats"
# print(data['AAPL']['advanced-stats']) to look for relevant endpoints.
# relevant endpoints: priceToBook, priceToSales
# print(data[symbol]['advanced-stats']['priceToBook']) for AAPL PBR
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#we will do the math ourselves for "enterprise value" and "gross profit"
# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [ ]:
Building our dataframe.

In [61]:
# rv stands for robust value.
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

#looping through every ticker within our within our symbol string and append 
#the data for that ticker to our empty pandas dataframe. 
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        # creating a try "except-statement" to handle type error from EBITDA 
        # this function tries to calculate ebitda and if it fails because of a type error
        #  then it will fill it in with np.NaN.
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        #N/A vlaues will be populated with percentiles shortly     
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [62]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128,N/A,54.15,N/A,8.23,N/A,7.24,N/A,35.050393,N/A,13.983073,N/A,N/A
1,AAL,21.81,N/A,-1.18,N/A,-2.05,N/A,0.8349,N/A,-4.327603,N/A,2.286952,N/A,N/A
2,AAP,165.6,N/A,22.6,N/A,3.08,N/A,1.11,N/A,11.290067,N/A,2.481896,N/A,N/A
3,AAPL,126.809,N/A,33.84,N/A,32.82,N/A,7.4,N/A,24.815618,N/A,19.216042,N/A,N/A
4,ABBV,111.77,N/A,40.97,N/A,15.14,N/A,4.34,N/A,14.136297,N/A,9.052075,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.5,N/A,36.97,N/A,-3.99,N/A,5.77,N/A,23.587143,N/A,9.685532,N/A,N/A
501,ZBH,164.7,N/A,-256.19,N/A,2.86,N/A,4.99,N/A,29.934454,N/A,8.596631,N/A,N/A
502,ZBRA,505.6,N/A,54.44,N/A,13.42,N/A,6.31,N/A,36.633125,N/A,14.054119,N/A,N/A
503,ZION,56.8,N/A,23.69,N/A,1.32,N/A,3.21,N/A,14.369808,N/A,3.727086,N/A,N/A


Handling missing values.
Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.
You can use pandas' isnull method to identify missing data:

In [57]:
# axis=1 indicated colummns not rows.
# funtion will return modified version of pandas dataframe that only contains
# contains rows where there's missing data
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,236.80,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,248.81,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.76,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,43.57,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.40,N/A,14.97,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,53.89,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,32.12,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,176.28,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2117.90,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.37,N/A,31.49,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [64]:
# funtion returns number of rows with missing data 
#len(rv_dataframe[rv_dataframe.isnull().any(axis=1)].index)

we will replace missing data with the average non-NaN data point from that column.

In [65]:
#replacing missing value with mean from column
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [66]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score



Calculating Value Percentiles
We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:
    Price-to-earnings ratio
    Price-to-book ratio
    Price-to-sales ratio
    EV/EBITDA
    EV/GP

Unlike before, we can't use the start of every string as a way to interpulate in that string because in the Momemtum project we had the same two words at the start of every momentum metric. ie. "one year price momentum" and "one year return percentiles". Checking above in the column names, you can't use "price to earnings ratio" name to find price to earnings ratio percentile column because there's an abreviation.   Instead, what we can do in copy our column names and turn them into a dictionary.     

In [71]:
# The way we're going to use this dictionar is that we're going to pass 
# in a metric and it will return back the name of the corresponding percentile
# To calculate our percentile scores we are going to loop through every metric in this
# in our metrics dictionary
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}
    
#test. should print 0-504
#for metric in metrics.keys():
#    for row in rv_dataframe.index:
#        print(row) 
    
for row in rv_dataframe.index:
    for metric in metrics.keys():                #entire column                                #values of that row in the column
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0       0.857426
1       0.148515
2       0.455446
3       0.681188
4       0.767327
         ...    
500     0.718812
501    0.0118812
502     0.859406
503     0.477228
504     0.825743
Name: PE Percentile, Length: 505, dtype: object
0       0.741584
1       0.049505
2       0.409901
3       0.956436
4       0.873267
         ...    
500    0.0475248
501     0.381188
502     0.855446
503     0.139604
504     0.925743
Name: PB Percentile, Length: 505, dtype: object
0            0.8
1      0.0732673
2       0.128713
3       0.805941
4       0.609901
         ...    
500     0.739604
501     0.691089
502     0.752475
503     0.472277
504     0.922772
Name: PS Percentile, Length: 505, dtype: object
0       0.883168
1      0.0336634
2       0.231683
3       0.770297
4       0.390099
         ...    
500     0.742574
501     0.851485
502     0.893069
503      0.39802
504     0.871287
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0           0.8
1       0.10099
2      0.108911
3    

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128,N/A,54.15,0.857426,8.23,0.741584,7.2400,0.8,35.050393,0.883168,13.983073,0.8,N/A
1,AAL,21.81,N/A,-1.18,0.148515,-2.05,0.049505,0.8349,0.0732673,-4.327603,0.0336634,2.286952,0.10099,N/A
2,AAP,165.6,N/A,22.60,0.455446,3.08,0.409901,1.1100,0.128713,11.290067,0.231683,2.481896,0.108911,N/A
3,AAPL,126.809,N/A,33.84,0.681188,32.82,0.956436,7.4000,0.805941,24.815618,0.770297,19.216042,0.916832,N/A
4,ABBV,111.77,N/A,40.97,0.767327,15.14,0.873267,4.3400,0.609901,14.136297,0.390099,9.052075,0.556436,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.5,N/A,36.97,0.718812,-3.99,0.0475248,5.7700,0.739604,23.587143,0.742574,9.685532,0.637624,N/A
501,ZBH,164.7,N/A,-256.19,0.0118812,2.86,0.381188,4.9900,0.691089,29.934454,0.851485,8.596631,0.542574,N/A
502,ZBRA,505.6,N/A,54.44,0.859406,13.42,0.855446,6.3100,0.752475,36.633125,0.893069,14.054119,0.80396,N/A
503,ZION,56.8,N/A,23.69,0.477228,1.32,0.139604,3.2100,0.472277,14.369808,0.39802,3.727086,0.186139,N/A


Recap:
we imported the percentile of score function from scipy stats module and we named it score. We created a dictionary of metrics for every key in the dictionary. It's one of the valuation metrics that we're using and every value of the dictionary is the corresponding percentile column of our pandas dataframe.

We created a double nested foreloop where the first foreloop loops over every metric ion our metric dictionary. And the second foreloop, loops over every row in our pandas dataframe. Within both of those foreloops we have an operation that calculates the percentile score of every valuation metric and assigns that value to our corresponding percentile score column. 

Now we need to loop through every rown in our padas dataframe and calculate that rv score for that row. Then access all of the percentile scores in our pandas dataframe and calculate their mean.

In [72]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128,N/A,54.15,0.857426,8.23,0.741584,7.2400,0.8,35.050393,0.883168,13.983073,0.8,0.816436
1,AAL,21.81,N/A,-1.18,0.148515,-2.05,0.049505,0.8349,0.0732673,-4.327603,0.0336634,2.286952,0.10099,0.0811881
2,AAP,165.6,N/A,22.60,0.455446,3.08,0.409901,1.1100,0.128713,11.290067,0.231683,2.481896,0.108911,0.266931
3,AAPL,126.809,N/A,33.84,0.681188,32.82,0.956436,7.4000,0.805941,24.815618,0.770297,19.216042,0.916832,0.826139
4,ABBV,111.77,N/A,40.97,0.767327,15.14,0.873267,4.3400,0.609901,14.136297,0.390099,9.052075,0.556436,0.639406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.5,N/A,36.97,0.718812,-3.99,0.0475248,5.7700,0.739604,23.587143,0.742574,9.685532,0.637624,0.577228
501,ZBH,164.7,N/A,-256.19,0.0118812,2.86,0.381188,4.9900,0.691089,29.934454,0.851485,8.596631,0.542574,0.495644
502,ZBRA,505.6,N/A,54.44,0.859406,13.42,0.855446,6.3100,0.752475,36.633125,0.893069,14.054119,0.80396,0.832871
503,ZION,56.8,N/A,23.69,0.477228,1.32,0.139604,3.2100,0.472277,14.369808,0.39802,3.727086,0.186139,0.334653


Selecting the best 50 stocks using pandas sorting 

In [73]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [74]:
# 50 cheapest stock
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.45,N/A,7.07,0.164356,0.5440,0.0554455,0.4479,0.0257426,3.529680,0.0415842,0.428754,0.0039604,0.0582178
1,MCK,174.28,N/A,-7.12,0.108911,-63.4300,0.00990099,0.1236,0.00594059,7.095875,0.0693069,2.770389,0.120792,0.0629703
2,FTI,8.9,N/A,-0.69,0.154455,0.8529,0.0712871,0.2568,0.00990099,2.225102,0.0376238,1.379856,0.0435644,0.0633663
3,AIG,45.04,N/A,-6.40,0.116832,0.6165,0.0594059,0.9093,0.0792079,7.376853,0.0772277,0.900135,0.0217822,0.0708911
4,ALL,111.85,N/A,6.39,0.162376,1.1900,0.106931,0.7328,0.0633663,2.359810,0.039604,0.748505,0.0158416,0.0776238
5,MET,58.3,N/A,10.64,0.194059,0.7108,0.0613861,0.7864,0.0693069,5.220724,0.0475248,0.795762,0.0178218,0.0780198
6,AAL,21.81,N/A,-1.18,0.148515,-2.0500,0.049505,0.8349,0.0732673,-4.327603,0.0336634,2.286952,0.10099,0.0811881
7,DXC,25.21,N/A,-2.27,0.136634,1.2000,0.111881,0.3707,0.0178218,5.286921,0.049505,2.325382,0.10495,0.0841584
8,ABC,104.66,N/A,-6.56,0.114851,-31.0300,0.0138614,0.1132,0.0039604,7.984418,0.10099,3.826710,0.190099,0.0847525
9,F,11.8,N/A,-37.97,0.0514851,1.5800,0.187129,0.3792,0.019802,6.319398,0.0554455,2.890061,0.122772,0.0873267


In [75]:
#Created from before
portfolio_input()
#print(portfolio_size)

Enter the value of your portfolio:2500000


Calculating position size

In [78]:
#position_size

50000.0

In [77]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.45,1821,7.07,0.164356,0.5440,0.0554455,0.4479,0.0257426,3.529680,0.0415842,0.428754,0.0039604,0.0582178
1,MCK,174.28,286,-7.12,0.108911,-63.4300,0.00990099,0.1236,0.00594059,7.095875,0.0693069,2.770389,0.120792,0.0629703
2,FTI,8.9,5617,-0.69,0.154455,0.8529,0.0712871,0.2568,0.00990099,2.225102,0.0376238,1.379856,0.0435644,0.0633663
3,AIG,45.04,1110,-6.40,0.116832,0.6165,0.0594059,0.9093,0.0792079,7.376853,0.0772277,0.900135,0.0217822,0.0708911
4,ALL,111.85,447,6.39,0.162376,1.1900,0.106931,0.7328,0.0633663,2.359810,0.039604,0.748505,0.0158416,0.0776238
5,MET,58.3,857,10.64,0.194059,0.7108,0.0613861,0.7864,0.0693069,5.220724,0.0475248,0.795762,0.0178218,0.0780198
6,AAL,21.81,2292,-1.18,0.148515,-2.0500,0.049505,0.8349,0.0732673,-4.327603,0.0336634,2.286952,0.10099,0.0811881
7,DXC,25.21,1983,-2.27,0.136634,1.2000,0.111881,0.3707,0.0178218,5.286921,0.049505,2.325382,0.10495,0.0841584
8,ABC,104.66,477,-6.56,0.114851,-31.0300,0.0138614,0.1132,0.0039604,7.984418,0.10099,3.826710,0.190099,0.0847525
9,F,11.8,4237,-37.97,0.0514851,1.5800,0.187129,0.3792,0.019802,6.319398,0.0554455,2.890061,0.122772,0.0873267


 Excel Output

In [79]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [80]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [81]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [82]:
writer.save()